In [ ]:
import os
import io
import re
import numpy as np
import pandas as pd
import pysam
from tqdm.notebook import tqdm 
from IPython.core.debugger import set_trace

In [ ]:
def read_vcf(path):
        with open(path, 'r') as f:
            lines = [l for l in f if not l.startswith('##')]
        res = None
        for bi in tqdm(range(int(np.ceil(len(lines)/1000)))):
            res_batch = pd.read_csv(io.StringIO(''.join([lines[0]] + lines[bi*1000 + 1:(bi+1)*1000])),
                dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
                       'QUAL': str, 'FILTER': str, 'INFO': str}, sep='\t')
            if res is None:
                if not res_batch[res_batch['#CHROM'] == '22'].empty:
                    res = res_batch[res_batch['#CHROM'] == '22']
            else:
                if not res_batch[res_batch['#CHROM'] == '22'].empty:
                    res = pd.concat([res, res_batch[res_batch['#CHROM'] == '22']])
        return res

foo_type = lambda x: pd.Series(x.split(';VC=')[1].split(';')[0])

# Read SNP databases

## dbSNP

whole genome avaiable at http://hgdownload.soe.ucsc.edu/goldenPath/hg19/database/snp151Common.txt.gz

here chr22 only: https://ftp.ncbi.nlm.nih.gov/snp/organisms/human_9606_b151_GRCh37p13/chr_rpts/

00-common_all.vcf

nb_lines = 37302979

In [ ]:
#dbsnp_df = read_vcf('../data/common_SNPs/00-common_all.vcf')
#print(dbsnp_df.shape)
#dbsnp_df.to_csv('../data/common_SNPs/dbsnp_df.csv', index=False)
#dbsnp_df.head()

In [ ]:
dbsnp_df = pd.read_csv('../data/common_SNPs/dbsnp_df.csv')
print(dbsnp_df.shape)
dbsnp_df.head()

In [ ]:
# type of SNP
dbsnp_df['type'] = dbsnp_df['INFO'].apply(foo_type)
dbsnp_df['type'].value_counts()

## genomAD database

In [ ]:
#genomad_df = read_vcf('../data/common_SNPs/gnomad.genomes.r2.1.1.sites.22.vcf')
#print(genomad_df.shape)
#genomad_df.head()

In [ ]:
#genomad_df.to_csv('../data/common_SNPs/genomad_df.csv')

In [ ]:
#genomad_df_new = pd.read_csv('../data/common_SNPs/genomad_df.csv', memory_map=True)
#genomad_df_new.head()

# Find reads supporting known SNPs in merged healthy bam

Important notes:
1. Indexing
    - SAM is a 1-index based file
    - VCF is a 1-index based file
    - pysam is a 0-index based tool

2. Paired-End sequencing
    - BAM/SAM are storing the resverse complementary of reversed reads as sequence

3. Mapping issues
    - some reads are not mapped -> no CIGAR string + no read.reference_end

In [ ]:
samfile = pysam.AlignmentFile("../data/healthy_chr22_merged-ready.bam", "rb")

# ititiate list of reads to remove
reads2remove = []
log_dict = {"position":[],"type":[],
            "total_reads":[], 'supporting_reads':[],
            "problematic_reads":[]}

# iterate over positions
for ci, mutation in tqdm(dbsnp_df.iloc[100000:105000].iterrows(), total=dbsnp_df.iloc[100000:150000].shape[0]):
    # set_trace()
    genotype = {'A': 0, 'C': 0, 'G': 0, 'T': 0, 'N': 0}
    print(mutation['type'], 'ref=', mutation['REF'], 'alt=', mutation['ALT'])
    c = 0 # number of reads supporting the considered mutation
    t = 0 # total number of reads at that position
    p = 0 # number pf reads with issues
    # iterate over reads that fall into the mutation position
    for read in samfile.fetch(str(mutation['#CHROM']), mutation['POS']-1, mutation['POS']): 
        t += 1
        seq = read.query_alignment_sequence
        pos = (mutation['POS'] -1) - read.reference_start + 1
        #print(pos-1, len(seq), mutation['REF'], mutation['ALT'], seq[pos-1], seq[max(pos-2, 0):min(pos+2, len(seq))])
        ######## SNV ##########
        if (mutation['type'] == 'SNV'):
            cigar = read.cigarstring
            if cigar is None:
                p += 1
            else:
                cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                cigar_states = re.split('[0-9]+',cigar)[1:]
                if 'D' in cigar_states:
                    if 'S' in cigar_states:
                        Spos = [i for i, x in enumerate(cigar_states) if x == "S"] 
                        cigar_pos = [i for j, i in enumerate(cigar_pos) if j not in Spos]
                        cigar_states = [i for j, i in enumerate(cigar_states) if j not in Spos]
                    Dpos = [i for i, x in enumerate(cigar_states) if x == "D"] 
                    for dpos in Dpos:
                        if sum(list(map(int, cigar_pos[:dpos]))) < pos:
                            pos = pos - int(cigar_pos[dpos])
            #print(len(seq), pos)
            genotype[seq[pos-1]] = genotype[seq[pos-1]]+1
            if ',' in mutation['ALT']:
                for muts in mutation['ALT'].split(','):
                    if (seq[pos-1:pos-1+1] == muts):
                        c += 1
                        reads2remove.append(read.query_name)
            else: 
                if (seq[pos-1:pos-1+1] == mutation['ALT']):
                    c += 1
                    reads2remove.append(read.query_name)

        elif (mutation['type'] == 'DIV'): # deletion or insertion
            ######## INSERTION ##########
            if len(mutation['ALT']) - len(mutation['REF']) > 0: # insertion
                # cond1 = nucleotide sequence comparison
                cond1 = False
                if ',' in mutation['ALT']:
                    for muts in mutation['ALT'].split(','):
                        if (seq[pos-1:pos-1+len(muts)] != muts):
                            cond1 = True
                else:
                    cond1 = (seq[pos-1:  pos-1+len(mutation['ALT'])] == mutation['ALT'])
                if cond1: 
                    # cond2 = cigar string indicates an insertion at this position
                    cigar = read.cigarstring
                    if cigar is None:
                        p += 1
                    else:
                        cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                        cigar_states = re.split('[0-9]+',cigar)[1:]
                        cond2 = False
                        if 'I' in cigar_states:
                            cumul = 0
                            indel_pos = None
                            for i, cp in enumerate(cigar_pos):
                                if (cigar_states[i] != 'S') and (cumul <= pos):
                                    cumul += -int(cp) if cp == 'D' else int(cp)
                                    indel_pos = i
                            if cigar_states[indel_pos] == 'I':
                                cond2 = True
                        if cond1 and cond2:
                            c += 1
                            reads2remove.append(read.query_name)
            ######## DELETION ##########
            elif len(mutation['ALT']) - len(mutation['REF']) < 0: # deletion
                # cond1 = nucleotide sequence comparison
                cond1 = False
                if ',' in mutation['ALT']:
                    for muts in mutation['ALT'].split(','):
                        if (seq[pos-1:pos-1+len(muts)] != muts):
                            cond1 = True
                else:
                    cond1 = (seq[pos-1:  pos-1+len(mutation['REF'])] != mutation['REF'])
                if cond1:
                    cigar = read.cigarstring
                    if cigar is None:
                        p += 1
                    else:
                        # cond2 = cigar string indicates a deletion at this position
                        cigar_pos = re.split('M|I|D|N|S|H|P|=|X',cigar)[:-1]
                        cigar_states = re.split('[0-9]+',cigar)[1:]
                        cond2 = False
                        if 'D' in cigar_states:
                            cumul = 0
                            indel_pos = None
                            for i, cp in enumerate(cigar_pos):
                                if (cigar_pos[i] != 'S') and (cumul <= pos):
                                    cumul += -int(cp) if cp == 'D' else int(cp)
                                    indel_pos = i
                            if cigar_states[indel_pos] == 'D':
                                cond2 = True
                        if cond1 and cond2:
                            c += 1
                            reads2remove.append(read.query_name)
    #if mutation['type'] == 'DIV':
    #print(t, c, p, mutation['type'], mutation['ALT'], mutation['REF'])
    log_dict["position"].append(mutation['POS'])
    log_dict["type"].append(mutation['type'])
    log_dict["total_reads"].append(t)
    log_dict["supporting_reads"].append(c)
    log_dict["problematic_reads"].append(p)
    print(genotype)

samfile.close()

In [ ]:
def read_vcf(path):
        with open(path, 'r') as f:
            lines = [l for l in f if not l.startswith('##')]
        res = pd.read_csv(io.StringIO(''.join(lines[:])),
            dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
                   'QUAL': str, 'FILTER': str, 'INFO': str}, sep='\t')
        return res

foo_vaf = lambda x: pd.Series(x.split(';AF=')[1].split(';')[0])

# Read SNPs detected in cancer patient

patient_snps = read_vcf('../data/2015-07-31_NCC_CRC-809_110914-CW/NCC_CRC-809_110914-CW-gatk-haplotype-annotated.vcf')
print(patient_snps.shape)
patient_snps = patient_snps[patient_snps['#CHROM'] == '22']
print(patient_snps.shape)
patient_snps.head()

patient_snps['VAF'] = patient_snps['INFO'].apply(foo_vaf)
patient_snps = patient_snps[['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'VAF']]
patient_snps

# print types of VAF
# heterozygote (=0.5), homozygote (1), double hetoerozygotes (0.5,0.5)
patient_snps['VAF'].value_counts()

In [ ]:
log_pd = pd.DataFrame.from_dict(log_dict)
log_pd

In [ ]:
log_pd[log_pd['total_reads'] > 0].shape[0]/log_pd.shape[0]

In [ ]:
100*log_pd[(log_pd['supporting_reads'] == 0) & (log_pd['total_reads'] > 0)].shape[0]/log_pd[log_pd['total_reads'] > 0].shape[0]

In [ ]:
print(vcf_pd.shape)

In [ ]:
a = 'GCAGCCACTCAGGATGTTGGAACCTGGCCATCCCTGCTTCTTTCAGTGGGTGAGGTTGGTGGCTGCTCCACCTGTTCCAGGCACACCCTTAACAGAGGTGGCTGCTTGCTCTTTAAGCCAGCTTGGCCTTGCCTGGCATGCACAGGCCCCG'
b = 'GCAGCCACTCAGGATGTTGGAACCTGGCCATCCCTGCTTCTTTCAGTGGGTGAGGTTGGTGGCTGCTCCACCTGTTCCAGGCACACCCTTAACAGAGGTGGCTGCTTGCTCTTTAAGCCAGCTTGGCCTTGCCTGGCATGCACAGGCCCCG'

a == b